## Этап 1

In [1]:
result = pd.read_csv("./reviews_keywords/wildberries_reviews_corrected.csv")
result.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 2937743 entries, 0 to 2937742
Data columns (total 7 columns):
 #   Column            Dtype
---  ------            -----
 0   Unnamed: 0        int64
 1   review_full_text  object
 2   review_rating     int64
 3   product           object
 4   category          object
 5   url               object
 6   corrected_text    object
dtypes: int64(2), object(5)
memory usage: 1.7+ GB


In [16]:
# Оставляем только по 5 записей для каждого уникального значения в столбце 'product'
result_limited = result.groupby('product').head(10000).reset_index(drop=True)
result_limited.describe()

,review_rating
count,2.937743e+06
mean,4.592586e+00
std,1.036270e+00
min,1.000000e+00
25%,5.000000e+00
50%,5.000000e+00
75%,5.000000e+00
max,5.000000e+00


In [2]:
import cudf.pandas  # Импортирование cuDF и активация его использования
cudf.pandas.install()  # Установка cuDF как основного интерфейса для pandas
import pandas as pd  # Импортирование pandas после установки cuDF

import os
import yaml
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from tqdm import tqdm
from IPython.display import display
import numpy as np

,corrected_text
count,517652
unique,331634
top,Мало мерит
freq,128


In [3]:
result = combined_df.merge(df_raw_big, left_index=True, right_index=True, how='right')
result.describe()

Map:   0%|          | 0/2999 [00:00<?, ? examples/s]

In [6]:
df_raw_big = None
combined_df = None

In [7]:
result['corrected_text'] = result['corrected_text'].fillna(result['review_full_text'])

In [8]:
# Оставляем только по 5 записей для каждого уникального значения в столбце 'product'
result_limited = result.groupby('product').head(10).reset_index(drop=True)
result_limited.describe()

Map:   0%|          | 0/3548 [00:00<?, ? examples/s]

In [3]:
import spacy
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.cluster import DBSCAN
import numpy as np
from collections import Counter

# Проверка доступности GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка модели и токенайзера от Сбербанка
tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/sbert_large_nlu_ru')
model = AutoModel.from_pretrained('sberbank-ai/sbert_large_nlu_ru').to(device)

# Загрузка и настройка модели SpaCy
nlp = spacy.load("ru_core_news_lg")

# Пример загрузки данных в pandas DataFrame
df_raw = pd.read_csv("wildberries_reviews.csv", nrows=30000)
df = df_raw[-3000:-1]  # Отбор 500 записей для обработки

# Преобразование pandas DataFrame в Hugging Face Dataset
dataset = Dataset.from_pandas(df)

import re

def clean_text(text):
    text = re.sub(r'[\n\r\t]+|\s{2,}', ' ', text)  # Объединяем шаги для замены пробелов
    text = re.sub(r'(?<!\.)\s*\.\s*|\s*\.\s*(?!\.)', '. ', text)  # Оптимизация замены точки
    return text.strip().rstrip('.')

def split_reviews_into_sentences(batch):
    # Очистка текстов
    cleaned_texts = [clean_text(text) for text in batch['corrected_text']]
    
    # Обработка текстов с помощью nlp.pipe с указанием batch_size
    docs = list(nlp.pipe(cleaned_texts, batch_size=64))  # Здесь 64 - пример значения

    # Извлечение предложений
    batch['sentences'] = [[sent.text for sent in doc.sents] for doc in docs]
    
    return batch

dataset = dataset.map(split_reviews_into_sentences, batched=True, batch_size=32)

# Преобразуем Dataset обратно в pandas DataFrame
df = dataset.to_pandas()

# Выполним explode по столбцу с предложениями
df_exploded = df.explode('sentences').reset_index(drop=True)

# Удаляем лишние столбцы, которые появились после explode
df_exploded = df_exploded.drop(columns=[col for col in df_exploded.columns if col.startswith('__index_level_')])

# Преобразуем DataFrame обратно в Hugging Face Dataset
dataset_exploded = Dataset.from_pandas(df_exploded)

from torch.cuda.amp import autocast

def compute_sentence_embeddings(sentences):
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        with autocast():  # Используем mixed precision для ускорения
            outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()


# Функция для вычисления эмбеддингов для каждого предложения после explode
def compute_embeddings_after_explode(batch):
    sentences = batch['sentences']
    embeddings = compute_sentence_embeddings(sentences)
    batch['sentence_embeddings'] = embeddings
    return batch

# Применение функции
dataset = dataset_exploded.map(compute_embeddings_after_explode, batched=True, batch_size=128)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [4]:
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_similarity
import torch
from tqdm import tqdm

# Устройство (GPU или CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Функция для вычисления центра кластера (центроида)
def find_centroid(embeddings):
    return np.mean(embeddings, axis=0)

# Функция для нахождения ключевой мысли в кластере
def extract_key_thought(cluster_sentences):
    sentences = cluster_sentences.split(" | ")
    embeddings = compute_sentence_embeddings(sentences)
    
    centroid = find_centroid(embeddings)
    similarities = cosine_similarity(embeddings, [centroid])
    key_sentence_index = np.argmax(similarities)
    
    return sentences[key_sentence_index]

# Функция для подсчета количества слов в каждом кластере
def count_words(cluster_sentences):
    words = cluster_sentences.split()
    return len(words)

# Функция для вычисления эмбеддингов
def compute_sentence_embeddings(sentences):
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
    return embeddings

# Функция для повторной кластеризации крупных кластеров
def recluster_large_cluster(cluster_sentences, eps=0.1, min_samples=2):
    sentences = cluster_sentences.split(" | ")
    
    embeddings = compute_sentence_embeddings(sentences)
    
    re_clustering = DBSCAN(eps=eps, min_samples=min_samples, metric="cosine").fit(embeddings)
    
    re_cluster_dict = {}
    for idx, label in enumerate(re_clustering.labels_):
        if label == -1:
            continue
        label_str = str(label)
        if label_str not in re_cluster_dict:
            re_cluster_dict[label_str] = []
        re_cluster_dict[label_str].append(sentences[idx])
    
    return [" | ".join(cluster) for cluster in re_cluster_dict.values()]

# Рекурсивная функция для кластеризации крупных кластеров
def recursive_clustering(cluster_sentences, threshold, eps=0.25, min_samples=3, min_eps=0.05):
    current_eps = eps
    new_clusters = [cluster_sentences]

    while True:
        next_clusters = []
        reclustered_any = False
        
        for cluster in new_clusters:
            if count_words(cluster) > threshold:
                while current_eps >= min_eps:
                    reclustered = recluster_large_cluster(cluster, eps=current_eps, min_samples=min_samples)
                    if len(reclustered) > 1:
                        next_clusters.extend(reclustered)
                        reclustered_any = True
                        break  # Кластер успешно разделен, выходим из внутреннего цикла
                    else:
                        current_eps *= 0.9  # Уменьшаем eps и пробуем снова
                
                if len(reclustered) == 1:
                    # Если кластер так и не был разделен, добавляем его обратно
                    next_clusters.append(cluster)
            else:
                next_clusters.append(cluster)
        
        new_clusters = next_clusters
        
        if not reclustered_any:
            break
    
    return new_clusters

# Основной процесс кластеризации по товарам
final_result = pd.DataFrame()

for product_name, group in df_exploded.groupby('product'):
    all_sentences = group['sentences'].tolist()

    # Обработка предложений без разделения на батчи
    all_embeddings = compute_sentence_embeddings(all_sentences)

    # Прогресс-бар для начальной кластеризации
    clustering = DBSCAN(eps=0.25, min_samples=3, metric="cosine").fit(all_embeddings)

    cluster_dict = {}
    for idx, label in tqdm(enumerate(clustering.labels_), desc=f"Organizing clusters for {product_name}"):
        if label == -1:
            continue
        label_str = str(label)
        if label_str not in cluster_dict:
            cluster_dict[label_str] = set()
        cluster_dict[label_str].add(all_sentences[idx])

    clusters = [" | ".join(sentences) for sentences in cluster_dict.values()]
    threshold = np.mean([count_words(cluster) for cluster in clusters]) * 1.5

    final_clusters = []
    for cluster in tqdm(clusters, desc="Recursive clustering"):
        final_clusters.extend(recursive_clustering(cluster, threshold))

    df_exploded_sorted = pd.DataFrame({'product': product_name, 'cluster_sentences': final_clusters})
    df_exploded_sorted['word_count'] = df_exploded_sorted['cluster_sentences'].apply(count_words)
    df_exploded_sorted['key_thought'] = df_exploded_sorted['cluster_sentences'].apply(extract_key_thought)

    df_exploded_sorted = df_exploded_sorted.sort_values(by='word_count', ascending=False)

    final_result = pd.concat([final_result, df_exploded_sorted], ignore_index=True)

# Показать результат
display(final_result[['product', 'cluster_sentences', 'key_thought', 'word_count']])


Обработка продуктов: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.06it/s]


,product,sentence,label,max_similarity
0,Shtapler / Лебедка электрическая 12v 3000lb 13...,Работает хорошо,-1,0.632879
1,Shtapler / Лебедка электрическая 12v 3000lb 13...,"Пришло быстро, все целое на вид.",1,0.535115
2,Shtapler / Лебедка электрическая 12v 3000lb 13...,Завтра буду испытывать,1,0.327607
3,Shtapler / Лебедка электрическая 12v 3000lb 13...,"Купил на квадр для поднятия отвала, установка ...",1,0.456785
4,Shtapler / Лебедка электрическая 12v 3000lb 13...,Лебёдка хорошая.,1,0.539105
...,...,...,...,...
361,Фрегат Лифт Подвеска / Лифт комплект рессоры К...,"На вид не плохие, но к сожалению на уаз профи ...",-1,0.560001
362,Фрегат Лифт Подвеска / Лифт комплект рессоры К...,Всё подошло,-1,0.609885
363,Фрегат Лифт Подвеска / Лифт комплект рессоры К...,"Все отлично, подошли без проблем!",0,0.696790
364,Фрегат Лифт Подвеска / Лифт комплект рессоры К...,Соответствует описанию!,-1,0.607369


In [5]:
# import os
# import numpy as np
# import pandas as pd
# from sklearn.cluster import DBSCAN
# from sklearn.metrics.pairwise import cosine_similarity
# import torch
# from transformers import AutoTokenizer, AutoModel
# import nltk
# from nltk.corpus import stopwords
# import spacy
# from tqdm import tqdm
# import logging

# # Отключение параллелизма в токенайзере Hugging Face
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

# # Устройство (GPU или CPU)
# device = torch.device("cpu")  # Ensure everything runs on CPU

# # Загрузка модели и токенайзера от Сбербанка
# tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/sbert_large_nlu_ru')
# model = AutoModel.from_pretrained('sberbank-ai/sbert_large_nlu_ru').to(device)

# # Настройка логирования
# logging.basicConfig(filename='./reviews_keywords/clustering.log', 
#                     level=logging.INFO, 
#                     format='%(asctime)s - %(levelname)s - %(message)s')

# # Загрузка модели spaCy для русского языка
# nlp = spacy.load("ru_core_news_lg")

# # Установка стоп-слов
# nltk.download('stopwords')
# stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
# # Функция для вычисления центра кластера (центроида)
# def find_centroid(embeddings):
#     return np.mean(embeddings, axis=0)

# # Функция для вычисления эмбеддингов
# def compute_sentence_embeddings(sentences):
#     inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to(device)
#     with torch.no_grad():
#         outputs = model(**inputs)
#         # Получаем скрытые состояния
#         hidden_states = outputs.hidden_states[-1]
#     embeddings = hidden_states.mean(dim=1).cpu().numpy()
#     return embeddings

# # Функция для нахождения ключевой мысли в кластере
# def extract_key_thought(cluster_sentences):
#     sentences = cluster_sentences.split(" | ")
    
#     embeddings = compute_sentence_embeddings(sentences)
    
#     centroid = find_centroid(embeddings)
#     similarities = cosine_similarity(embeddings, [centroid])
#     key_sentence_index = np.argmax(similarities)
    
#     return sentences[key_sentence_index]

# # Функция для подсчета количества слов в каждом кластере
# def count_words(cluster_sentences):
#     words = cluster_sentences.split()
#     return len(words)


# # Функция для повторной кластеризации крупных кластеров
# def recluster_large_cluster(cluster_sentences, eps=0.1, min_samples=2):
#     sentences = cluster_sentences.split(" | ")
    
#     embeddings = compute_sentence_embeddings(sentences)
    
#     re_clustering = DBSCAN(eps=eps, min_samples=min_samples, metric="cosine").fit(embeddings)
    
#     re_cluster_dict = {}
#     for idx, label in enumerate(re_clustering.labels_):
#         if label == -1:
#             continue
#         label_str = str(label)
#         if label_str not in re_cluster_dict:
#             re_cluster_dict[label_str] = []
#         re_cluster_dict[label_str].append(sentences[idx])
    
#     return [" | ".join(cluster) for cluster in re_cluster_dict.values()]

# # Рекурсивная функция для кластеризации крупных кластеров
# def recursive_clustering(cluster_sentences, threshold, eps=0.25, min_samples=3, min_eps=0.05):
#     current_eps = eps
#     new_clusters = [cluster_sentences]

#     while True:
#         next_clusters = []
#         reclustered_any = False
        
#         for cluster in new_clusters:
#             if count_words(cluster) > threshold:
#                 while current_eps >= min_eps:
#                     reclustered = recluster_large_cluster(cluster, eps=current_eps, min_samples=min_samples)
#                     if len(reclustered) > 1:
#                         next_clusters.extend(reclustered)
#                         reclustered_any = True
#                         break  # Кластер успешно разделен, выходим из внутреннего цикла
#                     else:
#                         current_eps -= 0.02  # Уменьшаем eps и пробуем снова
                
#                 if len(reclustered) == 1:
#                     # Если кластер так и не был разделен, добавляем его обратно
#                     next_clusters.append(cluster)
#             else:
#                 next_clusters.append(cluster)
        
#         new_clusters = next_clusters
        
#         if not reclustered_any:
#             break
    
#     return new_clusters

# # Основной процесс кластеризации по товарам
# df_clusters = pd.DataFrame()
# label_col = "label"
# sentence_col = "sentence"
# for label in final_result[label_col].unique():  # Added tqdm here
#     print(label)
#     label_df = final_result[final_result[label_col] == label]
#     all_sentences = label_df[sentence_col].tolist()
#     print(all_sentences)
#     # Обработка предложений без разделения на батчи
#     all_embeddings = compute_sentence_embeddings(all_sentences)
#     print(all_embeddings)

#     # Прогресс-бар для начальной кластеризации
#     clustering = DBSCAN(eps=0.25, min_samples=3, metric="cosine").fit(all_embeddings)
#     print(clustering)

#     cluster_dict = {}
#     for idx, label in tqdm(enumerate(clustering.labels_), desc=f"Organizing clusters for {label}"):
#         if label == -1:
#             continue
#         label_str = str(label)
#         if label_str not in cluster_dict:
#             cluster_dict[label_str] = set()
#         cluster_dict[label_str].add(all_sentences[idx])

#     clusters = [" | ".join(sentences) for sentences in cluster_dict.values()]
#     threshold = np.min([np.mean([count_words(cluster) for cluster in clusters]) * 1.5  ,  450])

#     final_clusters = []
#     for cluster in tqdm(clusters, desc="Recursive clustering"):
#         final_clusters.extend(recursive_clustering(cluster, threshold))

#     df_exploded_sorted = pd.DataFrame({'cluster_sentences': final_clusters})
#     df_exploded_sorted['word_count'] = df_exploded_sorted['cluster_sentences'].apply(count_words)
#     df_exploded_sorted['key_thought'] = df_exploded_sorted['cluster_sentences'].apply(extract_key_thought)

#     df_exploded_sorted = df_exploded_sorted.sort_values(by='word_count', ascending=False)

#     df_clusters = pd.concat([df_clusters, df_exploded_sorted], ignore_index=True)

# # Показать результат
# display(df_clusters[['cluster_sentences', 'key_thought', 'word_count']])


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


0
['Работает хорошо', 'Пришло быстро, все целое на вид.', 'Лебёдка хорошая.', 'Всё в комплекте, есть инструкция на русском языке', '👍', 'Не комплект, нет крюка, заметил поздно, кроме возврата ничего не предложили, проверяйте комплектность внимательнее, а не так как я', 'Деньги на ветер!', 'Товар понравился.', 'Только не доложили в комплекте одну гайку', 'Полный комплект для установки', 'При нажатии на кнопку отпускания, провод задымил.', 'Цена не три копейки', 'Все в комплекте работает', 'Очень хорошая лебедка', 'Дешево и сердито работает как положено', 'Полный комплект, поставил взамен сдохшей, товаром доволен!!!', 'Не комплект не доложили гак', 'Недорого, на мотобуксировщик то что надо.', 'Небольшой вес', 'Отличная', 'Все в комплекте спасибо советую', 'Просто крутая лебёдка!', 'Я очень доволен', 'Пришло быстро всё отлично', 'Качество нормальное', 'Все отлично, спасибо', '👍👍👍спасибо', 'Всё отлично', 'Пришло все целое.', 'Работает.', 'Муж доволен', 'Все хорошо работает', 'Отличная лебё

TypeError: 'NoneType' object is not subscriptable

## Этап 2

In [ ]:
import cudf.pandas  # Импортирование cuDF и активация его использования
cudf.pandas.install()  # Установка cuDF как основного интерфейса для pandas
import pandas as pd  # Импортирование pandas после установки cuDF

import os
import yaml
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from tqdm import tqdm
from IPython.display import display
import numpy as np

In [ ]:
# clustered_result = pd.read_csv("./reviews_keywords/cluster_result.csv").drop("Unnamed: 0", axis=1)

In [ ]:
# import re

# def extract_list_from_string(s):
#     # Регулярное выражение для поиска элементов списка внутри строки
#     matches = re.findall(r'\[\'(.*?)\'\]', s)
    
#     # Если нашли соответствие, разделяем элементы по запятой и возвращаем список
#     if matches:
#         return [item.strip() for item in matches[0].split("', '")]
#     return s

# # Применяем функцию ко всей колонке
# clustered_result['cluster_sentences'] = clustered_result['cluster_sentences'].apply(lambda x: extract_list_from_string(str(x)))

# # Проверяем результат
# clustered_result


In [ ]:
# # Группировка по метке `label` и объединение массивов текстов
# grouped_result = clustered_result.groupby('label').agg({
#     'cluster_sentences': lambda x: sum(x, []),  # Объединяем списки текстов
#     'cluster_id': 'first',  # Можно оставить любой cluster_id, так как они больше не будут уникальными
#     'key_thought': 'first',  # Оставляем первую ключевую мысль
#     'word_count': 'sum'  # Суммируем количество слов
# }).reset_index()
# # Подсчет итогового количества строк в массиве текстов
# grouped_result['total_sentences'] = grouped_result['cluster_sentences'].apply(len)
# grouped_result

In [ ]:
# grouped_result_exploded = grouped_result.explode('cluster_sentences')[['label', 'cluster_sentences']].drop_duplicates()
# grouped_result_exploded

In [ ]:
import cudf.pandas  # Импортирование cuDF и активация его использования
cudf.pandas.install()  # Установка cuDF как основного интерфейса для pandas
import pandas as pd  # Импортирование pandas после установки cuDF

import os
import yaml
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from tqdm import tqdm
from IPython.display import display
import numpy as np

In [ ]:
final_result = pd.read_csv("./reviews_keywords/final_result.csv")

In [45]:
final_result.label.value_counts()

label
 1    194
-1     87
 0     70
 2     15
Name: count, dtype: int64

In [48]:
final_result[final_result.label >= 0].label.value_counts()

label
1    194
0     70
2     15
Name: count, dtype: int64

In [ ]:
final_result.loc[final_result.label == 2, "label"] = 0
final_result = final_result[final_result.label >= 0]

In [ ]:
import re
import emoji
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from nltk.corpus import stopwords

# Установка стоп-слов
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

# Загрузка модели spaCy для русского языка
nlp = spacy.load("ru_core_news_lg")

# Функция для проверки наличия эмодзи в строке
def contains_emoji(text):
    return any(char in emoji.EMOJI_DATA for char in text)

# Существующие маски
common_phrases = [
    r'всё ок', r'супер', r'класс', r'нормально', r'норм', r'всё норм', r'отлично', r'хорошо', r'нормально упаковано',
    r'без проблем', r'как всегда', r'норм'
]
emotional_phrases = [
    r'спасибо', r'рекомендую', r'советую', r'продавец молодец', r'молодец', r'рекомендую продавца', r'благодарен', r'благодарю',
    r'советую к покупке', r'спасибо большое', r'всем советую'
]
short_phrases = [
    r'пришел быстро', r'уже брал', r'помогло', r'не помогло', r'пока не пробовал', r'отличная вещь', r'всё окей',
    r'нормально', r'быстрая доставка', r'пришел вовремя'
]
item_phrases = [
    r'хорошая вещь', r'классная вещь', r'отличная вещь', r'нужная вещь', r'удобная вещь', r'полезная вещь',
    r'прекрасная вещь', r'замечательная вещь', r'хороший продукт', r'отличный продукт', r'качественная вещь'
]
task_phrases = [
    r'с задачей справился', r'с функциями справился', r'задачу свою выполнил', r'справился на отлично', 
    r'функции выполняет', r'с задачей справляется', r'задачу выполнил', r'справляется с задачей', 
    r'со своими функциями справляется', r'справился с задачей'
]
delivery_phrases = [
    r'заказ пришел целый и вовремя', r'пришел вовремя', r'пришел целый', r'доставка вовремя', r'все пришло целым', 
    r'товар пришел целым', r'пришел в срок', r'доставка быстрая', r'пришел вовремя и целым', r'получил заказ вовремя'
]
emoji_phrases = [
    r'идеально', r'отлично', r'👍', r'👏', r'😆', r'🔥', r'💯', r'класс', r'класс👍', r'все супер👍', r'👍👍👍', r'👍😊'
]
negative_condition_phrases = [
    r'пришло все побитое', r'упаковка порвана', r'всё сломано', r'товар треснул', r'получил товар с дефектом', 
    r'погнутая упаковка', r'пришло разорванное', r'все разлито', r'коробка помята', r'всё побилось', 
    r'сломанный товар', r'все порвано', r'пришел весь в трещинах', r'поврежденная упаковка', r'товар не работает'
]
positive_condition_phrases = [
    r'всё пришло целое и невредимое', r'доставка - во!', r'крутая упаковка', r'упаковано на совесть', 
    r'все пришло в идеальном состоянии', r'товар в отличном состоянии', r'без повреждений', r'упаковка целая', 
    r'товар без дефектов', r'все пришло как надо', r'пришел в полном порядке', r'отличная упаковка', 
    r'все дошло целым', r'доставка без повреждений', r'идеальное состояние'
]
gratitude_phrases = [
    r'спасибо за товар', r'спасибо продавцу', r'спасибо большое', r'благодарю за товар', r'большое спасибо', 
    r'очень благодарен', r'спасибо за доставку', r'огромное спасибо', r'спасибо за качественный товар', 
    r'продавцу огромное спасибо', r'спасибо за оперативность', r'спасибо вам', r'благодарен за товар', 
    r'спасибо, всё хорошо', r'продавец молодец', r'спасибо за хорошее обслуживание'
]
neutral_quality_phrases = [
    r'всё отлично', r'всё хорошо', r'все супер', r'очень доволен покупкой', r'работает хорошо', 
    r'надеюсь прослужить долго', r'всё целое', r'всё в комплекте', r'всё как в описании', 
    r'всё как заявлено', r'за свою цену отлично', r'качество хорошее', r'отличное качество', 
    r'комплект как в описании', r'мелочь, а приятно', r'мне всё понравилось', r'добрый день', 
    r'всё соответствует', r'работает хорошо, спасибо', r'всё супер 👌'
]

# Новые маски
confirmation_phrases = [
    r'всё соответствует', r'всё как в описании', r'всё как заявлено', r'соответствует описанию', r'всё целое', r'всё в комплекте', r'всё норм', r'всё хорошо'
]
simple_statements_phrases = [
    r'хорошая вещь', r'классная вещь', r'отличная вещь', r'удобно', r'нормально', r'работает', r'работает отлично', r'работает хорошо', r'всё нормально', r'всё работает'
]
quality_phrases = [
    r'качество хорошее', r'отличное качество', r'качественно', r'прекрасное качество', r'высокое качество', r'качественный товар', r'качество отличное', r'качество удовлетворительное'
]
functionality_phrases = [
    r'работает отлично', r'работает хорошо', r'всё работает', r'функции выполняет', r'функциональный', r'функции справляются', r'с задачей справился', r'справляется с задачей', r'функции выполняет'
]
price_phrases = [
    r'цена нормальная', r'цена адекватная', r'соотношение цена/качество', r'цена отличная', r'цена хорошая', r'цена приемлемая', r'цена оправдана', r'цена низкая', r'цена высокая', r'соотношение цены и качества', r'за такую цену', r'вполне приемлемая цена'
]
durability_phrases = [
    r'надеюсь прослужить долго', r'пользуюсь долго', r'надежный товар', r'долговечный', r'хватит надолго', r'буду использовать долго', r'на сезон хватит', r'долго пользуюсь', r'проверено временем', r'выдерживает нагрузки', r'посмотрим, сколько продержится'
]
appearance_phrases = [
    r'выглядит хорошо', r'смотрится красиво', r'внешний вид отличный', r'стильно выглядит', r'выглядит красиво', r'смотрится отлично', r'внешне приятно', r'стильный', r'выглядит качественно'
]

# Функция для вычисления эмбеддингов
def compute_sentence_embeddings(sentences):
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

# Определение масок и их эмбеддингов
gratitude_emb = compute_sentence_embeddings(gratitude_phrases)
common_emb = compute_sentence_embeddings(common_phrases)
emotional_emb = compute_sentence_embeddings(emotional_phrases)
short_emb = compute_sentence_embeddings(short_phrases)
item_emb = compute_sentence_embeddings(item_phrases)
task_emb = compute_sentence_embeddings(task_phrases)
delivery_emb = compute_sentence_embeddings(delivery_phrases)
emoji_text_emb = compute_sentence_embeddings(emoji_phrases)
negative_condition_emb = compute_sentence_embeddings(negative_condition_phrases)
positive_condition_emb = compute_sentence_embeddings(positive_condition_phrases)
neutral_quality_emb = compute_sentence_embeddings(neutral_quality_phrases)
confirmation_emb = compute_sentence_embeddings(confirmation_phrases)
simple_statements_emb = compute_sentence_embeddings(simple_statements_phrases)
quality_emb = compute_sentence_embeddings(quality_phrases)
functionality_emb = compute_sentence_embeddings(functionality_phrases)
price_emb = compute_sentence_embeddings(price_phrases)
durability_emb = compute_sentence_embeddings(durability_phrases)
appearance_emb = compute_sentence_embeddings(appearance_phrases)

# Функция для проверки семантической близости с каждой маской
def is_similar_to_mask(key_thought, mask_emb):
    key_emb = compute_sentence_embeddings([key_thought])
    return np.max(cosine_similarity(key_emb, mask_emb)) > 0.65  # Порог близости можно настроить

# Проверка ключевых мыслей на семантическую близость к каждой маске
final_result['is_similar_to_emoji'] = final_result['key_thought'].apply(lambda x: contains_emoji(x) or is_similar_to_mask(x, emoji_text_emb))
final_result['is_similar_to_common'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, common_emb))
final_result['is_similar_to_emotional'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, emotional_emb))
final_result['is_similar_to_short'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, short_emb))
final_result['is_similar_to_item'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, item_emb))
final_result['is_similar_to_task'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, task_emb))
final_result['is_similar_to_delivery'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, delivery_emb))
final_result['is_similar_to_negative_condition'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, negative_condition_emb))
final_result['is_similar_to_positive_condition'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, positive_condition_emb))
final_result['is_similar_to_gratitude'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, gratitude_emb))
final_result['is_similar_to_neutral_quality'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, neutral_quality_emb))
final_result['is_similar_to_confirmation'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, confirmation_emb))
final_result['is_similar_to_simple_statements'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, simple_statements_emb))
final_result['is_similar_to_quality'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, quality_emb))
final_result['is_similar_to_functionality'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, functionality_emb))
final_result['is_similar_to_price'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, price_emb))
final_result['is_similar_to_durability'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, durability_emb))
final_result['is_similar_to_appearance'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, appearance_emb))

# Удаление пустых кластеров
final_result = final_result[final_result['cluster_sentences'].str.strip().astype(bool)]

# Слова для удаления кластеров
exclusion_words = [
    r'отличный', r'хороший', r'шикарный', r'офигенный', r'замечательный', r'потрясающий', r'великолепный', 
    r'прекрасный', r'изумительный', r'фантастический', r'удивительный', r'невероятный', r'зачётный', r'суперский', 
    r'классный', r'крутой', r'понравилось', r'понравились', r'люблю', r'восхищён', 
    r'доволен', r'наслаждаюсь', r'порадовало'
]

# Функция для лемматизации текста
def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])


# Предварительное вычисление эмбеддингов для лемматизированных слов из списка exclusion_words
lemmatized_exclusion_words = [lemmatize_text(word) for word in exclusion_words]
exclusion_emb = compute_sentence_embeddings(lemmatized_exclusion_words)

# Обновленная функция для проверки с использованием семантической близости
def is_single_word_or_stop_word(key_thought):
    words = re.findall(r'\w+', key_thought)  # Извлекаем все слова
    if len(words) == 1:
        return True
    if len(words) == 2 and words[1] in stop_words:
        return True
    if len(words) == 2 and re.match(r'[^\w\s]', words[1]):  # Пунктуация как второе слово
        return True
    if len(words) in [2, 3]:
        lemmatized_key_thought = lemmatize_text(key_thought)
        lemmatized_words = re.findall(r'\w+', lemmatized_key_thought)
        for word in lemmatized_words:
            key_emb = compute_sentence_embeddings([word])
            max_similarity = np.max(cosine_similarity(key_emb, exclusion_emb))
            if max_similarity > 0.9:  # Порог близости можно настроить
                print(f"Близость - {max_similarity}. Исключаем {key_thought}")
                return True
    return False

# Применение фильтрации
final_result = final_result[~final_result['key_thought'].apply(is_single_word_or_stop_word)]

# Обновление фильтрации кластеров, где все маски False
mask_false_clusters = (
    ~final_result['is_similar_to_emoji'] &
    ~final_result['is_similar_to_common'] &
    ~final_result['is_similar_to_emotional'] &
    ~final_result['is_similar_to_short'] &
    ~final_result['is_similar_to_item'] &
    ~final_result['is_similar_to_task'] &
    ~final_result['is_similar_to_delivery'] &
    ~final_result['is_similar_to_negative_condition'] &
    ~final_result['is_similar_to_positive_condition'] &
    ~final_result['is_similar_to_gratitude'] &
    ~final_result['is_similar_to_neutral_quality'] &
    ~final_result['is_similar_to_confirmation'] &
    ~final_result['is_similar_to_simple_statements'] &
    ~final_result['is_similar_to_quality'] &
    ~final_result['is_similar_to_functionality'] &
    ~final_result['is_similar_to_price'] &
    ~final_result['is_similar_to_durability'] &
    ~final_result['is_similar_to_appearance']
)

# Вывод результатов
df_false_clusters = final_result[mask_false_clusters]
display(df_false_clusters[['product', 'cluster_sentences', 'key_thought', 'word_count']])


,Unnamed: 0,label
count,555162.000000,555162.000000
mean,277580.500000,0.732846
std,160261.609409,0.442474
min,0.000000,0.000000
25%,138790.250000,0.000000
50%,277580.500000,1.000000
75%,416370.750000,1.000000
max,555161.000000,1.000000


In [11]:
df_false_clusters[['cluster_sentences', 'key_thought', 'word_count']].to_csv("./reviews_keywords/clusters.csv")

In [12]:
display(final_result.describe())
final_result.head()

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Summarizing clusters: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 62/62 [02:09<00:00,  2.09s/it]


,Unnamed: 0,product,sentence,label
0,0,!Крышка для микроволновой печи / Крышка для ми...,Оч понравилась!,1
1,1,!Крышка для микроволновой печи / Крышка для ми...,"Удобная ручка, плотный прозрачный материал - п...",1
2,2,!Крышка для микроволновой печи / Крышка для ми...,Поддон для воды удобный - нет необходимости ст...,1
3,3,!Крышка для микроволновой печи / Крышка для ми...,Под крышкой отлично видно разогревающуюся еду ...,1
4,4,!Крышка для микроволновой печи / Крышка для ми...,"Еще, как мне показалось, тарелка меньше нагрев...",1


In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from tqdm import tqdm

# Определение устройства (GPU или CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка модели T5
model_name = "cointegrated/rut5-base-multitask"  # Модель для русской T5
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Функция для разбиения текста на части
def chunk_text(text, max_length=100):
    words = text.split()
    chunks = [' '.join(words[i:i + max_length]) for i in range(0, len(words), max_length)]
    return chunks

# Функция для суммаризации текста с настройкой параметров генерации
def summarize_text(text):
    # Токенизация и перенос на GPU
    inputs = tokenizer("summarize: " + text, return_tensors="pt", max_length=512, truncation=True).to(device)
    
    # Генерация суммаризации с использованием GPU
    summary_ids = model.generate(
        inputs.input_ids, 
        max_length=150, 
        min_length=40, 
        length_penalty=4,  # Увеличиваем penalty для избежания повторений
        num_beams=16,  # Увеличиваем количество beam для улучшения качества
        repetition_penalty=3.0,  # Добавляем штраф за повторения
        early_stopping=True
    )
    
    # Перенос результата обратно на CPU и декодирование
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Функция для суммаризации длинных текстов с рекурсивным подходом
def recursive_summarization(text, depth=2):
    chunks = chunk_text(text, max_length=100)  # Разбиение текста на части, каждая до 100 слов
    summaries = [summarize_text(chunk) for chunk in chunks]
    
    # Если достигли необходимой глубины рекурсии, возвращаем результат
    if depth <= 1:
        return ' '.join(summaries)
    
    # В противном случае суммаризируем еще раз на более высокой глубине
    return recursive_summarization(' '.join(summaries), depth - 1)

# Применение рекурсивной суммаризации к каждому кластеру с прогресс-баром и использованием GPU
df_false_clusters['summary'] = [
    recursive_summarization(text, depth=2) for text in tqdm(df_false_clusters['cluster_sentences'], desc="Summarizing clusters")
]

# Вывод результатов суммаризации
display(df_false_clusters[['cluster_sentences', 'summary']])


2024-08-19 22:55:55.247994: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-19 22:55:55.275809: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-19 22:55:55.878923: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/sbert_large_nlu_ru and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
10,0.628100,0.645104,0.245136,0.253846,0.485294,0.171875


In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Получаем предсказания модели на валидационном наборе данных
outputs = trainer.predict(val_dataset)
logits = outputs.predictions
labels = outputs.label_ids

# Функция для вычисления метрик для различных порогов
def evaluate_thresholds(logits, labels, thresholds):
    best_threshold = 0
    best_f1 = 0
    best_metrics = {}
    
    for threshold in thresholds:
        predictions = (logits[:, 1] > threshold).astype(int)
        precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
        acc = accuracy_score(labels, predictions)
        
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
            best_metrics = {
                'accuracy': acc,
                'f1': f1,
                'precision': precision,
                'recall': recall,
                'threshold': threshold
            }
    
    return best_metrics

# Диапазон порогов, который будем проверять
thresholds = np.arange(0.1, 0.9, 0.05)

# Вычисление метрик для различных порогов
best_metrics = evaluate_thresholds(logits, labels, thresholds)

# Вывод лучших метрик и оптимального порога
print(f"Лучший порог: {best_metrics['threshold']}")
print(f"Точность: {best_metrics['accuracy']}")
print(f"F1: {best_metrics['f1']}")
print(f"Precision: {best_metrics['precision']}")
print(f"Recall: {best_metrics['recall']}")


Лучший порог: 0.40000000000000013
Точность: 0.8949416342412452
F1: 0.9298701298701298
Precision: 0.927461139896373
Recall: 0.9322916666666666


## Кластеризация и всякие тесты

In [ ]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Загрузка модели и токенайзера для коррекции текста
model_name = "cointegrated/rut5-base-multitask"  # Модель T5 для мультитаскинга на русском языке
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# Функция для удаления повторов и объединения текста
def clean_text(text):
    sentences = text.split('<br>')
    cleaned_sentences = []
    seen = set()
    
    for sentence in sentences:
        sentence = sentence.strip()
        if sentence not in seen:
            cleaned_sentences.append(sentence)
            seen.add(sentence)
    
    return ' '.join(cleaned_sentences)

# Функция для корректировки текста с использованием T5
def correct_text(text):
    inputs = tokenizer("correct: " + text, return_tensors="pt", max_length=512, truncation=True).to(device)
    summary_ids = model.generate(inputs.input_ids, max_length=150, min_length=40, length_penalty=1.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Пример текста
text = 'Хороший очиститель, чистил дроссель старой Тойоты, отмывает очень хорошо, спасибо продавцу. <br>Очень хорошо отмывает загрязнения, нагар Хороший очиститель, прешёл хорошо упакован, спасибо продавцу <br>Хороший герметик, помог. <br>Хороший герметик, помог. <br>Отличный товар рекомендую'

# Очистка текста от повторов
cleaned_text = clean_text(text)

# Корректировка текста для улучшения согласованности и пунктуации
final_text = correct_text(cleaned_text)
final_text


In [ ]:
df_exploded_sorted["cluster_sentences"].to_csv("./reviews_keywords/clusters.csv")

In [ ]:
# import re
# import emoji
# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np
# import torch
# from transformers import AutoTokenizer, AutoModel
# from nltk.corpus import stopwords

# # Установка стоп-слов
# nltk.download('stopwords')
# stop_words = set(stopwords.words('russian'))

# # Загрузка модели spaCy для русского языка
# nlp = spacy.load("ru_core_news_lg")

# # Функция для проверки наличия эмодзи в строке
# def contains_emoji(text):
#     return any(char in emoji.EMOJI_DATA for char in text)

# # Существующие маски
# common_phrases = [
#     r'всё ок', r'супер', r'класс', r'нормально', r'норм', r'всё норм', r'отлично', r'хорошо', r'нормально упаковано',
#     r'без проблем', r'как всегда', r'норм'
# ]
# emotional_phrases = [
#     r'спасибо', r'рекомендую', r'советую', r'продавец молодец', r'молодец', r'рекомендую продавца', r'благодарен', r'благодарю',
#     r'советую к покупке', r'спасибо большое', r'всем советую'
# ]
# short_phrases = [
#     r'пришел быстро', r'уже брал', r'помогло', r'не помогло', r'пока не пробовал', r'отличная вещь', r'всё окей',
#     r'нормально', r'быстрая доставка', r'пришел вовремя'
# ]
# item_phrases = [
#     r'хорошая вещь', r'классная вещь', r'отличная вещь', r'нужная вещь', r'удобная вещь', r'полезная вещь',
#     r'прекрасная вещь', r'замечательная вещь', r'хороший продукт', r'отличный продукт', r'качественная вещь'
# ]
# task_phrases = [
#     r'с задачей справился', r'с функциями справился', r'задачу свою выполнил', r'справился на отлично', 
#     r'функции выполняет', r'с задачей справляется', r'задачу выполнил', r'справляется с задачей', 
#     r'со своими функциями справляется', r'справился с задачей'
# ]
# delivery_phrases = [
#     r'заказ пришел целый и вовремя', r'пришел вовремя', r'пришел целый', r'доставка вовремя', r'все пришло целым', 
#     r'товар пришел целым', r'пришел в срок', r'доставка быстрая', r'пришел вовремя и целым', r'получил заказ вовремя'
# ]
# emoji_phrases = [
#     r'идеально', r'отлично', r'👍', r'👏', r'😆', r'🔥', r'💯', r'класс', r'класс👍', r'все супер👍', r'👍👍👍', r'👍😊'
# ]
# negative_condition_phrases = [
#     r'пришло все побитое', r'упаковка порвана', r'всё сломано', r'товар треснул', r'получил товар с дефектом', 
#     r'погнутая упаковка', r'пришло разорванное', r'все разлито', r'коробка помята', r'всё побилось', 
#     r'сломанный товар', r'все порвано', r'пришел весь в трещинах', r'поврежденная упаковка', r'товар не работает'
# ]
# positive_condition_phrases = [
#     r'всё пришло целое и невредимое', r'доставка - во!', r'крутая упаковка', r'упаковано на совесть', 
#     r'все пришло в идеальном состоянии', r'товар в отличном состоянии', r'без повреждений', r'упаковка целая', 
#     r'товар без дефектов', r'все пришло как надо', r'пришел в полном порядке', r'отличная упаковка', 
#     r'все дошло целым', r'доставка без повреждений', r'идеальное состояние'
# ]
# gratitude_phrases = [
#     r'спасибо за товар', r'спасибо продавцу', r'спасибо большое', r'благодарю за товар', r'большое спасибо', 
#     r'очень благодарен', r'спасибо за доставку', r'огромное спасибо', r'спасибо за качественный товар', 
#     r'продавцу огромное спасибо', r'спасибо за оперативность', r'спасибо вам', r'благодарен за товар', 
#     r'спасибо, всё хорошо', r'продавец молодец', r'спасибо за хорошее обслуживание'
# ]
# neutral_quality_phrases = [
#     r'всё отлично', r'всё хорошо', r'все супер', r'очень доволен покупкой', r'работает хорошо', 
#     r'надеюсь прослужить долго', r'всё целое', r'всё в комплекте', r'всё как в описании', 
#     r'всё как заявлено', r'за свою цену отлично', r'качество хорошее', r'отличное качество', 
#     r'комплект как в описании', r'мелочь, а приятно', r'мне всё понравилось', r'добрый день', 
#     r'всё соответствует', r'работает хорошо, спасибо', r'всё супер 👌'
# ]

# # Новые маски
# confirmation_phrases = [
#     r'всё соответствует', r'всё как в описании', r'всё как заявлено', r'соответствует описанию', r'всё целое', r'всё в комплекте', r'всё норм', r'всё хорошо'
# ]
# simple_statements_phrases = [
#     r'хорошая вещь', r'классная вещь', r'отличная вещь', r'удобно', r'нормально', r'работает', r'работает отлично', r'работает хорошо', r'всё нормально', r'всё работает'
# ]
# quality_phrases = [
#     r'качество хорошее', r'отличное качество', r'качественно', r'прекрасное качество', r'высокое качество', r'качественный товар', r'качество отличное', r'качество удовлетворительное'
# ]
# functionality_phrases = [
#     r'работает отлично', r'работает хорошо', r'всё работает', r'функции выполняет', r'функциональный', r'функции справляются', r'с задачей справился', r'справляется с задачей', r'функции выполняет'
# ]
# price_phrases = [
#     r'цена нормальная', r'цена адекватная', r'соотношение цена/качество', r'цена отличная', r'цена хорошая', r'цена приемлемая', r'цена оправдана', r'цена низкая', r'цена высокая', r'соотношение цены и качества', r'за такую цену', r'вполне приемлемая цена'
# ]
# durability_phrases = [
#     r'надеюсь прослужить долго', r'пользуюсь долго', r'надежный товар', r'долговечный', r'хватит надолго', r'буду использовать долго', r'на сезон хватит', r'долго пользуюсь', r'проверено временем', r'выдерживает нагрузки', r'посмотрим, сколько продержится'
# ]
# appearance_phrases = [
#     r'выглядит хорошо', r'смотрится красиво', r'внешний вид отличный', r'стильно выглядит', r'выглядит красиво', r'смотрится отлично', r'внешне приятно', r'стильный', r'выглядит качественно'
# ]

# # Функция для вычисления эмбеддингов
# def compute_sentence_embeddings(sentences):
#     inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to(device)
#     with torch.no_grad():
#         outputs = model(**inputs)
#     return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

# # Определение масок и их эмбеддингов
# gratitude_emb = compute_sentence_embeddings(gratitude_phrases)
# common_emb = compute_sentence_embeddings(common_phrases)
# emotional_emb = compute_sentence_embeddings(emotional_phrases)
# short_emb = compute_sentence_embeddings(short_phrases)
# item_emb = compute_sentence_embeddings(item_phrases)
# task_emb = compute_sentence_embeddings(task_phrases)
# delivery_emb = compute_sentence_embeddings(delivery_phrases)
# emoji_text_emb = compute_sentence_embeddings(emoji_phrases)
# negative_condition_emb = compute_sentence_embeddings(negative_condition_phrases)
# positive_condition_emb = compute_sentence_embeddings(positive_condition_phrases)
# neutral_quality_emb = compute_sentence_embeddings(neutral_quality_phrases)
# confirmation_emb = compute_sentence_embeddings(confirmation_phrases)
# simple_statements_emb = compute_sentence_embeddings(simple_statements_phrases)
# quality_emb = compute_sentence_embeddings(quality_phrases)
# functionality_emb = compute_sentence_embeddings(functionality_phrases)
# price_emb = compute_sentence_embeddings(price_phrases)
# durability_emb = compute_sentence_embeddings(durability_phrases)
# appearance_emb = compute_sentence_embeddings(appearance_phrases)

# # Функция для проверки семантической близости с каждой маской
# def is_similar_to_mask(key_thought, mask_emb):
#     key_emb = compute_sentence_embeddings([key_thought])
#     return np.max(cosine_similarity(key_emb, mask_emb)) > 0.65  # Порог близости можно настроить

# # Проверка ключевых мыслей на семантическую близость к каждой маске
# final_result['is_similar_to_emoji'] = final_result['key_thought'].apply(lambda x: contains_emoji(x) or is_similar_to_mask(x, emoji_text_emb))
# final_result['is_similar_to_common'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, common_emb))
# final_result['is_similar_to_emotional'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, emotional_emb))
# final_result['is_similar_to_short'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, short_emb))
# final_result['is_similar_to_item'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, item_emb))
# final_result['is_similar_to_task'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, task_emb))
# final_result['is_similar_to_delivery'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, delivery_emb))
# final_result['is_similar_to_negative_condition'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, negative_condition_emb))
# final_result['is_similar_to_positive_condition'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, positive_condition_emb))
# final_result['is_similar_to_gratitude'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, gratitude_emb))
# final_result['is_similar_to_neutral_quality'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, neutral_quality_emb))
# final_result['is_similar_to_confirmation'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, confirmation_emb))
# final_result['is_similar_to_simple_statements'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, simple_statements_emb))
# final_result['is_similar_to_quality'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, quality_emb))
# final_result['is_similar_to_functionality'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, functionality_emb))
# final_result['is_similar_to_price'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, price_emb))
# final_result['is_similar_to_durability'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, durability_emb))
# final_result['is_similar_to_appearance'] = final_result['key_thought'].apply(lambda x: is_similar_to_mask(x, appearance_emb))

# # Удаление пустых кластеров
# final_result = final_result[final_result['cluster_sentences'].str.strip().astype(bool)]

# # Слова для удаления кластеров
# exclusion_words = [
#     r'отличный', r'хороший', r'шикарный', r'офигенный', r'замечательный', r'потрясающий', r'великолепный', 
#     r'прекрасный', r'изумительный', r'фантастический', r'удивительный', r'невероятный', r'зачётный', r'суперский', 
#     r'классный', r'крутой', r'понравилось', r'понравились', r'люблю', r'восхищён', 
#     r'доволен', r'наслаждаюсь', r'порадовало'
# ]

# # Функция для лемматизации текста
# def lemmatize_text(text):
#     doc = nlp(text)
#     return " ".join([token.lemma_ for token in doc])


# # Предварительное вычисление эмбеддингов для лемматизированных слов из списка exclusion_words
# lemmatized_exclusion_words = [lemmatize_text(word) for word in exclusion_words]
# exclusion_emb = compute_sentence_embeddings(lemmatized_exclusion_words)

# # Обновленная функция для проверки с использованием семантической близости
# def is_single_word_or_stop_word(key_thought):
#     words = re.findall(r'\w+', key_thought)  # Извлекаем все слова
#     if len(words) == 1:
#         return True
#     if len(words) == 2 and words[1] in stop_words:
#         return True
#     if len(words) == 2 and re.match(r'[^\w\s]', words[1]):  # Пунктуация как второе слово
#         return True
#     if len(words) in [2, 3]:
#         lemmatized_key_thought = lemmatize_text(key_thought)
#         lemmatized_words = re.findall(r'\w+', lemmatized_key_thought)
#         for word in lemmatized_words:
#             key_emb = compute_sentence_embeddings([word])
#             max_similarity = np.max(cosine_similarity(key_emb, exclusion_emb))
#             if max_similarity > 0.9:  # Порог близости можно настроить
#                 print(f"Близость - {max_similarity}. Исключаем {key_thought}")
#                 return True
#     return False

# # Применение фильтрации
# final_result = final_result[~final_result['key_thought'].apply(is_single_word_or_stop_word)]

# # Обновление фильтрации кластеров, где все маски False
# mask_false_clusters = (
#     ~final_result['is_similar_to_emoji'] &
#     ~final_result['is_similar_to_common'] &
#     ~final_result['is_similar_to_emotional'] &
#     ~final_result['is_similar_to_short'] &
#     ~final_result['is_similar_to_item'] &
#     ~final_result['is_similar_to_task'] &
#     ~final_result['is_similar_to_delivery'] &
#     ~final_result['is_similar_to_negative_condition'] &
#     ~final_result['is_similar_to_positive_condition'] &
#     ~final_result['is_similar_to_gratitude'] &
#     ~final_result['is_similar_to_neutral_quality'] &
#     ~final_result['is_similar_to_confirmation'] &
#     ~final_result['is_similar_to_simple_statements'] &
#     ~final_result['is_similar_to_quality'] &
#     ~final_result['is_similar_to_functionality'] &
#     ~final_result['is_similar_to_price'] &
#     ~final_result['is_similar_to_durability'] &
#     ~final_result['is_similar_to_appearance']
# )

# # Вывод результатов
# df_false_clusters = final_result[mask_false_clusters]
# display(df_false_clusters[['product', 'cluster_sentences', 'key_thought', 'word_count']])


In [ ]:
# df_false_clusters[['cluster_sentences', 'key_thought', 'word_count']].to_csv("./reviews_keywords/clusters.csv")

In [ ]:
# import torch
# from transformers import T5ForConditionalGeneration, T5Tokenizer
# from tqdm import tqdm

# # Определение устройства (GPU или CPU)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Загрузка модели T5
# model_name = "cointegrated/rut5-base-multitask"  # Модель для русской T5
# model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
# tokenizer = T5Tokenizer.from_pretrained(model_name)

# # Функция для разбиения текста на части
# def chunk_text(text, max_length=100):
#     words = text.split()
#     chunks = [' '.join(words[i:i + max_length]) for i in range(0, len(words), max_length)]
#     return chunks

# # Функция для суммаризации текста с настройкой параметров генерации
# def summarize_text(text):
#     # Токенизация и перенос на GPU
#     inputs = tokenizer("summarize: " + text, return_tensors="pt", max_length=512, truncation=True).to(device)
    
#     # Генерация суммаризации с использованием GPU
#     summary_ids = model.generate(
#         inputs.input_ids, 
#         max_length=150, 
#         min_length=40, 
#         length_penalty=4,  # Увеличиваем penalty для избежания повторений
#         num_beams=16,  # Увеличиваем количество beam для улучшения качества
#         repetition_penalty=3.0,  # Добавляем штраф за повторения
#         early_stopping=True
#     )
    
#     # Перенос результата обратно на CPU и декодирование
#     return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# # Функция для суммаризации длинных текстов с рекурсивным подходом
# def recursive_summarization(text, depth=2):
#     chunks = chunk_text(text, max_length=100)  # Разбиение текста на части, каждая до 100 слов
#     summaries = [summarize_text(chunk) for chunk in chunks]
    
#     # Если достигли необходимой глубины рекурсии, возвращаем результат
#     if depth <= 1:
#         return ' '.join(summaries)
    
#     # В противном случае суммаризируем еще раз на более высокой глубине
#     return recursive_summarization(' '.join(summaries), depth - 1)

# # Применение рекурсивной суммаризации к каждому кластеру с прогресс-баром и использованием GPU
# df_false_clusters['summary'] = [
#     recursive_summarization(text, depth=2) for text in tqdm(df_false_clusters['cluster_sentences'], desc="Summarizing clusters")
# ]

# # Вывод результатов суммаризации
# display(df_false_clusters[['cluster_sentences', 'summary']])


In [ ]:
# import re
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# # Загрузка модели и токенайзера для коррекции текста
# model_name = "cointegrated/rut5-base-multitask"  # Модель T5 для мультитаскинга на русском языке
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# # Функция для удаления повторов и объединения текста
# def clean_text(text):
#     sentences = text.split('<br>')
#     cleaned_sentences = []
#     seen = set()
    
#     for sentence in sentences:
#         sentence = sentence.strip()
#         if sentence not in seen:
#             cleaned_sentences.append(sentence)
#             seen.add(sentence)
    
#     return ' '.join(cleaned_sentences)

# # Функция для корректировки текста с использованием T5
# def correct_text(text):
#     inputs = tokenizer("correct: " + text, return_tensors="pt", max_length=512, truncation=True).to(device)
#     summary_ids = model.generate(inputs.input_ids, max_length=150, min_length=40, length_penalty=1.0, num_beams=4, early_stopping=True)
#     return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# # Пример текста
# text = 'Хороший очиститель, чистил дроссель старой Тойоты, отмывает очень хорошо, спасибо продавцу. <br>Очень хорошо отмывает загрязнения, нагар Хороший очиститель, прешёл хорошо упакован, спасибо продавцу <br>Хороший герметик, помог. <br>Хороший герметик, помог. <br>Отличный товар рекомендую'

# # Очистка текста от повторов
# cleaned_text = clean_text(text)

# # Корректировка текста для улучшения согласованности и пунктуации
# final_text = correct_text(cleaned_text)
# final_text


In [ ]:
# df_exploded_sorted["cluster_sentences"].to_csv("./reviews_keywords/clusters.csv")

In [ ]:
# def filter_tokens(syntax_analysis):
#     # Отключаем некоторые фильтры для проверки
#     filtered_tokens = [
#         token for token in syntax_analysis 
#         if token[1] not in {"PUNCT", "SPACE"}  # Исключаем только знаки препинания и пробелы
#         # Отключаем фильтрацию по длине
#     ]
    
#     return filtered_tokens

# def extract_key_phrases_from_sentences(doc):
#     key_phrases = []
    
#     for sent in doc.sents:
#         syntax_analysis = [(token.text, token.pos_, token.dep_, token.head.text) for token in sent]
#         filtered_tokens = filter_tokens(syntax_analysis)
#         phrase = []

#         for i, token in enumerate(filtered_tokens):
#             if token[1] in {"NOUN", "VERB"}:  # Существительное или глагол
#                 if phrase:
#                     key_phrases.append(" ".join(phrase))
#                     phrase = []
#                 phrase.append(token[0])
#             elif token[1] in {"ADJ", "ADV"}:  # Прилагательные, наречия
#                 if phrase:
#                     phrase.append(token[0])

#             # Если конец текста или следующая часть речи не связана с текущей фразой
#             if i == len(filtered_tokens) - 1 or filtered_tokens[i+1][1] not in {"ADJ", "ADV", "ADP", "CCONJ", "SCONJ", "PART"}:
#                 if phrase:
#                     key_phrases.append(" ".join(phrase))
#                     phrase = []
#     key_phrases = [phrase for phrase in key_phrases if len(phrase.split()) > 1 and len(phrase.strip()) > 2]

#     return " ".join(key_phrases)


# def extract_key_phrases_from_clusters(clusters):
#     key_phrases = []
#     for cluster in clusters:
#         cluster_key_phrases = []
#         for sentences in cluster:  # Так как cluster теперь список списков
#             doc = nlp(sentences)
#             cluster_key_phrases.append(extract_key_phrases_from_sentences(doc))
#         key_phrases.append(" ".join(cluster_key_phrases))  # Соединяем все ключевые фразы из одного кластера в одну строку
#     return key_phrases

# # Применение функции
# dataset = dataset.map(lambda batch: {"key_phrases": extract_key_phrases_from_clusters(batch['clusters'])}, batched=True, batch_size=8)


# # Частотный анализ по ключевым фразам
# key_phrases = dataset['key_phrases']
# phrase_freq = Counter(key_phrases)

# # Вывод результатов
# print("Частотный анализ ключевых фраз (по семантической близости):")
# print(phrase_freq.most_common(10))

In [ ]:
# df.to_csv("./reviews_keywords/temp_spacy.csv", index=False)